In [2]:
import re
import ast
import pandas as pd
from pymongo import MongoClient
import datetime
import pickle
import psycopg2

def unix_to_beijing_hour(timestamp):
    beijing_offset = datetime.timedelta(hours=8)
    beijing_time = datetime.datetime.utcfromtimestamp(timestamp) + beijing_offset
    return beijing_time.replace(minute=0, second=0).strftime("%H")

def count_elements(mmap):
    count = 0
    for num in mmap.values():
        count += num
    return count


In [3]:



# MongoDB连接参数
mongo_host = 'law.conetop.cn'
mongo_port = 27017
mongo_database_name = 'harbintrips'

# 创建MongoDB客户端并连接到数据库
mongo_client = MongoClient(host=mongo_host, port=mongo_port)
mongo_db = mongo_client[mongo_database_name]





mmap = dict()

# 从MongoDB中读取数据并处理
for num in range(3, 8):
    collection_name = f'trips2_{num:02}'
    collection = mongo_db[collection_name]
    cursor = collection.find()
    while True:
        try:
            doc = cursor.next()
            data = pd.DataFrame([doc])  # 将单个文档转换为 DataFrame

            x = data.iloc[0]

            # 以下处理逻辑与之前相同
            if (x['latitudes'] is None or len(x['latitudes']) == 0) or (x['longitudes'] is None or len(x['longitudes']) == 0) or (x['time'] is None or len(x['time']) == 0):
                continue

            latitude = round(x['latitudes'][0], 2)
            longitude = round(x['longitudes'][0], 2)
            time = unix_to_beijing_hour(x['time'][0])

            if (latitude, longitude) not in mmap:
                mmap[(latitude, longitude)] = {}

            if time not in mmap[(latitude, longitude)]:
                mmap[(latitude, longitude)][time] = 0

            mmap[(latitude, longitude)][time] += 1

            # 以(latitude,longitude)为圆心，四万平方米（六个标准足球场）内出现点单的数量
            print(f'当前该地区{(latitude, longitude)}共有({count_elements(mmap[(latitude, longitude)])})个订单', end=" -> ")
            print(f'时间：{time} 共有', end=' -> ')
            print(f'{mmap[(latitude, longitude)][time]} 个订单')
        except StopIteration:
            print('time error')
            break
            
    print(f'\n\n\n{collection_name}, over\n\n\n')



当前该地区(45.75, 126.69)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.75, 126.75)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.71, 126.68)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.75, 126.65)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.77, 126.7)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.72, 126.53)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.74, 126.61)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.75, 126.63)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.77, 126.63)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.77, 126.65)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.76, 126.68)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.74, 126.66)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.73, 126.63)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.79, 126.65)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.74, 126.63)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.72, 126.62)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.73, 126.59)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.75, 126.62)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.74, 126.59)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.76, 126.64)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当

当前该地区(45.76, 126.67)共有(6)个订单 -> 时间：00 共有 -> 6 个订单
当前该地区(45.75, 126.59)共有(3)个订单 -> 时间：00 共有 -> 3 个订单
当前该地区(45.79, 126.68)共有(4)个订单 -> 时间：00 共有 -> 4 个订单
当前该地区(45.75, 126.66)共有(8)个订单 -> 时间：00 共有 -> 8 个订单
当前该地区(45.75, 126.66)共有(9)个订单 -> 时间：00 共有 -> 9 个订单
当前该地区(45.78, 126.66)共有(2)个订单 -> 时间：00 共有 -> 2 个订单
当前该地区(45.81, 126.53)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.76, 126.6)共有(4)个订单 -> 时间：00 共有 -> 4 个订单
当前该地区(45.77, 126.63)共有(2)个订单 -> 时间：00 共有 -> 2 个订单
当前该地区(45.73, 126.67)共有(5)个订单 -> 时间：00 共有 -> 5 个订单
当前该地区(45.73, 126.69)共有(3)个订单 -> 时间：00 共有 -> 3 个订单
当前该地区(45.79, 126.68)共有(5)个订单 -> 时间：00 共有 -> 5 个订单
当前该地区(45.75, 126.67)共有(7)个订单 -> 时间：00 共有 -> 7 个订单
当前该地区(45.75, 126.63)共有(8)个订单 -> 时间：00 共有 -> 8 个订单
当前该地区(45.7, 126.69)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.74, 126.7)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.79, 126.65)共有(5)个订单 -> 时间：00 共有 -> 5 个订单
当前该地区(45.78, 126.63)共有(2)个订单 -> 时间：00 共有 -> 2 个订单
当前该地区(45.74, 126.67)共有(3)个订单 -> 时间：00 共有 -> 3 个订单
当前该地区(45.75, 126.61)共有(9)个订单 -> 时间：00 共有 -> 9 个订单
当前该

当前该地区(45.78, 126.69)共有(3)个订单 -> 时间：00 共有 -> 3 个订单
当前该地区(45.76, 126.66)共有(7)个订单 -> 时间：00 共有 -> 7 个订单
当前该地区(45.74, 126.7)共有(2)个订单 -> 时间：00 共有 -> 2 个订单
当前该地区(45.75, 126.62)共有(9)个订单 -> 时间：00 共有 -> 9 个订单
当前该地区(45.69, 126.66)共有(2)个订单 -> 时间：00 共有 -> 2 个订单
当前该地区(45.78, 126.61)共有(2)个订单 -> 时间：00 共有 -> 2 个订单
当前该地区(45.76, 126.63)共有(15)个订单 -> 时间：00 共有 -> 15 个订单
当前该地区(45.72, 126.66)共有(2)个订单 -> 时间：00 共有 -> 2 个订单
当前该地区(45.71, 126.62)共有(2)个订单 -> 时间：00 共有 -> 2 个订单
当前该地区(45.8, 126.55)共有(2)个订单 -> 时间：00 共有 -> 2 个订单
当前该地区(45.83, 126.56)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.75, 126.67)共有(12)个订单 -> 时间：00 共有 -> 12 个订单
当前该地区(45.77, 126.65)共有(10)个订单 -> 时间：00 共有 -> 10 个订单
当前该地区(45.77, 126.63)共有(7)个订单 -> 时间：00 共有 -> 7 个订单
当前该地区(45.75, 126.67)共有(13)个订单 -> 时间：00 共有 -> 13 个订单
当前该地区(45.77, 126.65)共有(11)个订单 -> 时间：00 共有 -> 11 个订单
当前该地区(45.79, 126.69)共有(6)个订单 -> 时间：00 共有 -> 6 个订单
当前该地区(45.73, 126.61)共有(7)个订单 -> 时间：00 共有 -> 7 个订单
当前该地区(45.74, 126.62)共有(11)个订单 -> 时间：00 共有 -> 11 个订单
当前该地区(45.78, 126.67)共有(7)个订单 -> 时间：00 共有

当前该地区(45.72, 126.63)共有(9)个订单 -> 时间：00 共有 -> 9 个订单
当前该地区(45.73, 126.63)共有(9)个订单 -> 时间：00 共有 -> 9 个订单
当前该地区(45.76, 126.71)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.75, 126.66)共有(12)个订单 -> 时间：00 共有 -> 12 个订单
当前该地区(45.73, 126.61)共有(9)个订单 -> 时间：00 共有 -> 9 个订单
当前该地区(45.76, 126.65)共有(3)个订单 -> 时间：00 共有 -> 3 个订单
当前该地区(45.75, 126.69)共有(14)个订单 -> 时间：00 共有 -> 14 个订单
当前该地区(45.71, 126.66)共有(4)个订单 -> 时间：00 共有 -> 4 个订单
当前该地区(45.73, 126.58)共有(4)个订单 -> 时间：00 共有 -> 4 个订单
当前该地区(45.79, 126.69)共有(9)个订单 -> 时间：00 共有 -> 9 个订单
当前该地区(45.74, 126.59)共有(17)个订单 -> 时间：00 共有 -> 17 个订单
当前该地区(45.76, 126.61)共有(13)个订单 -> 时间：00 共有 -> 13 个订单
当前该地区(45.77, 126.64)共有(10)个订单 -> 时间：00 共有 -> 10 个订单
当前该地区(45.78, 126.65)共有(28)个订单 -> 时间：00 共有 -> 28 个订单
当前该地区(45.69, 126.64)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.77, 126.65)共有(16)个订单 -> 时间：00 共有 -> 16 个订单
当前该地区(45.75, 126.73)共有(1)个订单 -> 时间：00 共有 -> 1 个订单
当前该地区(45.71, 126.64)共有(4)个订单 -> 时间：00 共有 -> 4 个订单
当前该地区(45.74, 126.63)共有(7)个订单 -> 时间：00 共有 -> 7 个订单
当前该地区(45.73, 126.62)共有(7)个订单 -> 时间：0

当前该地区(45.76, 126.6)共有(16)个订单 -> 时间：00 共有 -> 16 个订单
当前该地区(45.69, 126.63)共有(4)个订单 -> 时间：00 共有 -> 4 个订单
当前该地区(45.75, 126.63)共有(19)个订单 -> 时间：00 共有 -> 19 个订单
当前该地区(45.74, 126.63)共有(14)个订单 -> 时间：00 共有 -> 14 个订单
当前该地区(45.74, 126.68)共有(10)个订单 -> 时间：00 共有 -> 10 个订单
当前该地区(45.74, 126.68)共有(11)个订单 -> 时间：00 共有 -> 11 个订单
当前该地区(45.77, 126.66)共有(8)个订单 -> 时间：00 共有 -> 8 个订单
当前该地区(45.72, 126.52)共有(2)个订单 -> 时间：00 共有 -> 2 个订单
当前该地区(45.72, 126.58)共有(4)个订单 -> 时间：00 共有 -> 4 个订单
当前该地区(45.72, 126.68)共有(8)个订单 -> 时间：00 共有 -> 8 个订单
当前该地区(45.73, 126.58)共有(10)个订单 -> 时间：00 共有 -> 10 个订单
当前该地区(45.76, 126.64)共有(23)个订单 -> 时间：00 共有 -> 23 个订单
当前该地区(45.74, 126.6)共有(16)个订单 -> 时间：00 共有 -> 16 个订单
当前该地区(45.78, 126.65)共有(33)个订单 -> 时间：00 共有 -> 33 个订单
当前该地区(45.71, 126.68)共有(4)个订单 -> 时间：00 共有 -> 4 个订单
当前该地区(45.77, 126.71)共有(2)个订单 -> 时间：00 共有 -> 2 个订单
当前该地区(45.74, 126.63)共有(15)个订单 -> 时间：00 共有 -> 15 个订单
当前该地区(45.76, 126.61)共有(16)个订单 -> 时间：00 共有 -> 16 个订单
当前该地区(45.79, 126.68)共有(7)个订单 -> 时间：00 共有 -> 7 个订单
当前该地区(45.72, 126.56)共有(6)个订单 -

KeyboardInterrupt: 

In [ ]:
sorted_mmap = dict(sorted(mmap.items(), key=lambda x: count_elements(x[1]), reverse=True))

# 保存 mmap 字典
with open('sorted_mmap.pickle', 'wb') as f:  # 以二进制写入模式打开文件
    pickle.dump(sorted_mmap, f)  # 将 mmap 字典保存到文件
    
mongo_client.close()

In [3]:
sorted_mmap = {}
with open('sorted_mmap.pickle', 'rb') as f:  # 以二进制读取模式打开文件
    sorted_mmap = pickle.load(f)  # 从文件中加载 mmap 字典

In [4]:
cc = 0
for key, inner_dict in sorted_mmap.items():
    count = count_elements(inner_dict)
    cc+=count
    print(f'Key: {key}, Element Count: {count}')
print(cc)

Key: (45.74, 126.66), Element Count: 32162
Key: (45.7, 126.61), Element Count: 29531
Key: (45.76, 126.63), Element Count: 26017
Key: (45.75, 126.6), Element Count: 21738
Key: (45.76, 126.66), Element Count: 20199
Key: (45.78, 126.65), Element Count: 19680
Key: (45.77, 126.64), Element Count: 18315
Key: (45.75, 126.68), Element Count: 17383
Key: (45.76, 126.64), Element Count: 16857
Key: (45.77, 126.65), Element Count: 16683
Key: (45.74, 126.59), Element Count: 15669
Key: (45.72, 126.6), Element Count: 15579
Key: (45.75, 126.61), Element Count: 15439
Key: (45.77, 126.62), Element Count: 15258
Key: (45.76, 126.61), Element Count: 15208
Key: (45.76, 126.6), Element Count: 15143
Key: (45.7, 126.57), Element Count: 14541
Key: (45.71, 126.66), Element Count: 14432
Key: (45.75, 126.7), Element Count: 14398
Key: (45.73, 126.7), Element Count: 14026
Key: (45.73, 126.67), Element Count: 13750
Key: (45.74, 126.62), Element Count: 13635
Key: (45.75, 126.69), Element Count: 13631
Key: (45.73, 126.6

In [17]:


# PostgreSQL连接参数
pg_host = 'law.conetop.cn'  # 替换为实际的主机地址
pg_port = 5432  # 通常PostgreSQL默认端口是5432，可根据实际修改
pg_database_name = 'harbintrips'  # 替换为实际数据库名称
pg_user = 'zhongtai'  # 替换为你的用户名
pg_password = 'zt123!'  # 替换为你的密码

# 建立与PostgreSQL数据库的连接
pg_conn = psycopg2.connect(
    host=pg_host,
    port=pg_port,
    database=pg_database_name,
    user=pg_user,
    password=pg_password
)
pg_cursor = pg_conn.cursor()

# 创建PostgreSQL表（如果不存在），设计表结构来存放处理后的数据
create_table_query = """
CREATE TABLE IF NOT EXISTS processed_data (
    location text,
    time_00 integer,
    time_01 integer,
    time_02 integer,
    time_03 integer,
    time_04 integer,
    time_05 integer,
    time_06 integer,
    time_07 integer,
    time_08 integer,
    time_09 integer,
    time_10 integer,
    time_11 integer,
    time_12 integer,
    time_13 integer,
    time_14 integer,
    time_15 integer,
    time_16 integer,
    time_17 integer,
    time_18 integer,
    time_19 integer,
    time_20 integer,
    time_21 integer,
    time_22 integer,
    time_23 integer,
    PRIMARY KEY (location)
);
"""
pg_cursor.execute(create_table_query)

# # 将处理后的数据（mmap字典）存入PostgreSQL数据库
# for location, time_dict in sorted_mmap.items():
#     insert_query = "INSERT INTO processed_data (location"
#     values = [location]
#     for time in time_dict.keys():
#         insert_query += f", time_{time}"
#         values.append(time_dict[time])
#     insert_query += ") VALUES (%s" + ", %s" * len(time_dict) + ")"
#     pg_cursor.execute(insert_query, tuple(values))

sorted_mmap_list = list(sorted_mmap.items())    
for index in range(100):
    location, time_dict = sorted_mmap_list[index]
    insert_query = "INSERT INTO processed_data (location"
    values = [location]
    for time in time_dict.keys():
        insert_query += f", time_{time}"
        values.append(time_dict[time])
    insert_query += ") VALUES (%s" + ", %s" * len(time_dict) + ")"
    pg_cursor.execute(insert_query, tuple(values))

# 提交事务，使插入操作生效
pg_conn.commit()

# 关闭游标和连接
pg_cursor.close()
pg_conn.close()

print("数据已成功从MongoDB读取并存储到PostgreSQL中。")

数据已成功从MongoDB读取并存储到PostgreSQL中。


In [24]:
# 建立与PostgreSQL数据库的连接
pg_conn = psycopg2.connect(
    host=pg_host,
    port=pg_port,
    database=pg_database_name,
    user=pg_user,
    password=pg_password
)
pg_cursor = pg_conn.cursor()

# 以下是从PostgreSQL读取并打印数据的代码部分
# 编写查询语句，这里查询所有数据，你可以根据实际需求修改查询条件
select_query = "SELECT * FROM processed_data"
# 执行查询语句
pg_cursor.execute(select_query)
# 获取查询结果
results = pg_cursor.fetchall()

# 遍历结果，计算并打印每个location中time_XX数据资源的和
for row in results:
    location = row[0]
    time_sum = 0
    for i in range(1, 24):  # 对应time_00到time_23字段的索引范围
        value = row[i]
        if value is not None:  # 判断值是否为None，只有不为None时才进行累加
            time_sum += value
    print(f"Location: {location}, Time Data Sum: {time_sum}")

# 关闭游标和连接
pg_cursor.close()
pg_conn.close()

Location: (45.74,126.66), Time Data Sum: 31898
Location: (45.7,126.61), Time Data Sum: 29123
Location: (45.76,126.63), Time Data Sum: 25250
Location: (45.75,126.6), Time Data Sum: 21135
Location: (45.76,126.66), Time Data Sum: 19703
Location: (45.78,126.65), Time Data Sum: 18777
Location: (45.77,126.64), Time Data Sum: 18072
Location: (45.75,126.68), Time Data Sum: 17031
Location: (45.76,126.64), Time Data Sum: 16619
Location: (45.77,126.65), Time Data Sum: 16313
Location: (45.74,126.59), Time Data Sum: 15288
Location: (45.72,126.6), Time Data Sum: 15279
Location: (45.75,126.61), Time Data Sum: 14776
Location: (45.77,126.62), Time Data Sum: 14997
Location: (45.76,126.61), Time Data Sum: 14824
Location: (45.76,126.6), Time Data Sum: 14797
Location: (45.7,126.57), Time Data Sum: 14030
Location: (45.71,126.66), Time Data Sum: 14158
Location: (45.75,126.7), Time Data Sum: 14134
Location: (45.73,126.7), Time Data Sum: 13370
Location: (45.73,126.67), Time Data Sum: 13222
Location: (45.74,126

In [16]:
# # PostgreSQL连接参数
# pg_host = 'law.conetop.cn'  # 替换为实际的主机地址
# pg_port = 5432  # 通常PostgreSQL默认端口是5432，可根据实际修改
# pg_database_name = 'harbintrips'  # 替换为实际数据库名称
# pg_user = 'zhongtai'  # 替换为你的用户名
# pg_password = 'zt123!'  # 替换为你的用户名对应的密码

# # 建立与PostgreSQL数据库的连接
# pg_conn = psycopg2.connect(
#     host=pg_host,
#     port=pg_port,
#     database=pg_database_name,
#     user=pg_user,
#     password=pg_password
# )
# pg_cursor = pg_conn.cursor()

# # 编写删除表的SQL语句
# drop_table_query = "DROP TABLE IF EXISTS processed_data"
# # 执行删除表的语句
# pg_cursor.execute(drop_table_query)

# # 提交事务，使删除操作生效（即使表不存在，执行此语句也不会报错）
# pg_conn.commit()

# # 关闭游标和连接
# pg_cursor.close()
# pg_conn.close()

# print("表已成功删除（如果存在）。")

表已成功删除（如果存在）。
